In [ ]:
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [ ]:
# Load and clean the data
data = pd.read_csv('properties_with_distances.csv')
data.dropna(subset=['meter_sale_price'], inplace=True)  # Ensure there are no NaN values in target variable

Берем только продажи

In [ ]:
processing_data = data.loc[data['trans_group_id'] == 1][['meter_sale_price', 'property_type_en', 'instance_date', 'property_sub_type_en', 'property_usage_en', 'rooms_en', 'procedure_area', 'has_parking', 'distance_to_center', 'area_name_en']]

area_rating = pd.read_csv('residential_area_rating.csv')

processing_data = processing_data.merge(area_rating, on='area_name_en', how='left')

processing_data['instance_date'] = pd.to_datetime(processing_data['instance_date'], format='%d-%m-%Y')
processing_data.set_index('instance_date', inplace=True)

processing_data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
model_data = processing_data[processing_data.index > '2000-01-01']
processing_data.index = pd.DatetimeIndex(processing_data.index).to_period('M')

# model_data = model_data[model_data['meter_sale_price'] < 175000]
processing_data

In [ ]:
sarimax_residential_data = model_data[model_data['property_usage_en'] == 'Residential'][['meter_sale_price', 'property_type_en', 'rooms_en', 'has_parking', 'distance_to_center', 'area_rating']]

def room_count(value):
    mapping = {
        '1 B/R': 1,
        '2 B/R': 2,
        'Studio': 1,
        '3 B/R': 3,
        '4 B/R': 4,
        '5 B/R': 5,
        'PENTHOUSE': 10,
        'Single Room': 1,
        '6 B/R': 6,
        'Shop': 0
    }
    return mapping.get(value, 0) 

sarimax_residential_data['rooms'] = sarimax_residential_data['rooms_en'].apply(room_count)

sarimax_residential_data

In [ ]:
sarimax_full_data = sarimax_residential_data[['meter_sale_price', 'rooms', 'has_parking', 'distance_to_center', 'area_rating']][sarimax_residential_data.index > '2000-01-01']

sarimax_full_data

In [ ]:

# Plotting
plt.figure(figsize=(12, 8))
sns.lineplot(data=sarimax_full_data['meter_sale_price'])
plt.title('Meter Sale Price Over Time')
plt.xlabel('Date')
plt.ylabel('Average Meter Sale Price')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

len(sarimax_full_data)

In [ ]:
from statsmodels.tsa.stattools import kpss

def kpss_test(series, **kw):
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'Number of Lags: {n_lags}')
    print('Critical Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    return statistic, p_value, n_lags, critical_values


In [ ]:
cap_value = sarimax_full_data['meter_sale_price'].quantile(0.95)
sarimax_full_data = sarimax_full_data[sarimax_full_data['meter_sale_price'] < cap_value]


print(cap_value)
len(sarimax_full_data)

In [ ]:
# Plot data without outliers
plt.figure(figsize=(14, 8))
sns.lineplot(data=sarimax_full_data['meter_sale_price'])
plt.title('Meter Sale Price Over Time (Without Outliers)')
plt.xlabel('Date')
plt.ylabel('Average Meter Sale Price')
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

# sarimax_data = pd.get_dummies(sarimax_residential_data[['meter_sale_price', 'property_type_en', 'rooms', 'has_parking', 'distance_to_center']], columns=['property_type_en'], drop_first=False)
sarimax_data = sarimax_full_data[['meter_sale_price', 'rooms', 'has_parking', 'distance_to_center', 'area_rating']].resample('ME').mean().dropna()
# 
# 
# Plotting
plt.figure(figsize=(12, 8))
sns.lineplot(data=sarimax_data['meter_sale_price'])
plt.title('Average Meter Sale Price Over Time')
plt.xlabel('Date')
plt.ylabel('Average Meter Sale Price')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


plot_acf(sarimax_data['meter_sale_price'].dropna())
plot_pacf(sarimax_data['meter_sale_price'].dropna())
plt.show()

result = adfuller(sarimax_data['meter_sale_price'])
print('ADF Statistic:', result[0])
print('p-value:', result[1])
if result[1] < 0.05:
    print("Series is stationary")
else:
    print("Series is not stationary, differencing might be required")

    
kpss_test(sarimax_data['meter_sale_price'])

sarimax_data

In [ ]:

sarimax_data = sarimax_full_data[['meter_sale_price', 'rooms', 'has_parking', 'distance_to_center', 'area_rating']].resample('ME').mean().dropna()

# Plotting
plt.figure(figsize=(12, 8))
sns.lineplot(data=sarimax_data['meter_sale_price'].diff().dropna())
plt.title('Average Meter Sale Diff. Price Over Time')
plt.xlabel('Date')
plt.ylabel('Average Meter Sale Price')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

 
plot_acf(sarimax_data['meter_sale_price'].diff().dropna())
plot_pacf(sarimax_data['meter_sale_price'].diff().dropna())
plt.show()

result = adfuller(sarimax_data['meter_sale_price'].diff().dropna())
print('ADF Statistic:', result[0])
print('p-value:', result[1])
if result[1] < 0.05:
    print("Series is stationary")
else:
    print("Series is not stationary, differencing might be required")
    
sarimax_data

In [ ]:
kpss_test(sarimax_data['meter_sale_price'].diff().dropna())

In [ ]:
sarimax_full_data['year'] = sarimax_full_data.index.year
# 
gdp = pd.read_csv('gdp.csv', sep=';')
gdp.set_index('year', inplace=True)
gdp = gdp['GDP'].to_dict()

trade_balance = pd.read_csv('trade_balance.csv')
trade_balance.set_index('year', inplace=True)
trade_balance = trade_balance['balance'].to_dict()

inflation = pd.read_csv('inflation.csv')
inflation.set_index('year', inplace=True)
inflation = inflation['inflation'].to_dict()

exchange_rate = pd.read_csv('aed_to_usd.csv')
exchange_rate.set_index('year_month', inplace=True)
exchange_rate = exchange_rate['rate'].to_dict()

oil = pd.read_csv('Crude Oil WTI Futures Historical Data.csv')
oil.set_index('year_month', inplace=True)
oil = oil['Price'].to_dict()

sarimax_full_data['year_month'] = sarimax_full_data.index.strftime('%Y-%m')

sarimax_full_data['gdp'] = sarimax_full_data['year'].apply(lambda row: gdp[row] / 10 ** 9)
sarimax_full_data['trade_balance'] = sarimax_full_data['year'].apply(lambda row: trade_balance[row])
sarimax_full_data['inflation'] = sarimax_full_data['year'].apply(lambda row: inflation[row])
sarimax_full_data['aed_to_usd'] = sarimax_full_data.apply(lambda row: exchange_rate[row['year_month']], axis=1)
sarimax_full_data['oil'] = sarimax_full_data.apply(lambda row: oil[row['year_month']], axis=1)

sarimax_data = sarimax_full_data[[
    'meter_sale_price', 
    'rooms', 
    'has_parking', 
    'distance_to_center', 
    'area_rating', 
    'gdp', 
    'inflation',
    'oil',
    'aed_to_usd',
    'trade_balance',
]].resample('ME').mean().dropna()
# # 

# 
sarimax_data

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm

train = sarimax_data[[
    'meter_sale_price', 
    'rooms', 
    'has_parking', 
    'distance_to_center', 
    'area_rating', 
    'gdp', 
    # 'inflation',
    # 'oil',
    # 'aed_to_usd',
    # 'trade_balance',
]]

exog_train = train.drop(columns=['meter_sale_price'], axis=1)

model = sm.tsa.SARIMAX(train['meter_sale_price'], order=(1, 1, 1), seasonal_order=(0, 0, 0, 0), exog=exog_train)
fitted_model = model.fit(method='nm', maxiter=1000, disp=False)

summary = fitted_model.summary()

coef_table = summary.tables[1].data[1:]

coef_data = []
for row in coef_table:
    variable = row[0]
    coef = float(row[1])
    std_err = float(row[2])
    z_value = float(row[3])
    p_value = float(row[4])
    t_value = z_value  # для больших выборок z-статистики приблизительно равны t-статистикам
    coef_data.append([variable, coef, std_err, t_value, p_value])

results_df = pd.DataFrame(coef_data, columns=['Variable', 'Coefficient', 'Std.Err.', 't', 'P>|t|'])

# Добавление звездочек для значимости
def significance_stars(p):
    if p < 0.01:
        return '***'
    elif p < 0.05:
        return '**'
    elif p < 0.1:
        return '*'
    else:
        return ''

results_df['Significance'] = results_df['P>|t|'].apply(significance_stars)
results_df = results_df[['Variable', 'Coefficient', 'Std.Err.', 't', 'P>|t|', 'Significance']]

# Печать таблицы
print(results_df)

# Предсказанные значения
y_pred = fitted_model.fittedvalues

# Фактические значения
y_true = train['meter_sale_price']

# Рассчитаем R^2
ss_res = np.sum((y_true - y_pred) ** 2)
ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
r_squared = 1 - (ss_res / ss_tot)

print(f"R^2 (within): {r_squared:.4f}")

print(summary)

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm

train = sarimax_data[[
    'meter_sale_price', 
    'rooms', 
    'has_parking', 
    'distance_to_center', 
    'area_rating', 
    'gdp', 
]].iloc[:-12]
test = sarimax_data[[
    'meter_sale_price', 
    'rooms', 
    'has_parking', 
    'distance_to_center', 
    'area_rating', 
    'gdp', 
]].iloc[-12:]

exog_train = train.drop(columns=['meter_sale_price'], axis=1)
exog_test = test.drop(columns=['meter_sale_price'], axis=1)

model = sm.tsa.SARIMAX(train['meter_sale_price'], order=(1, 1, 1), seasonal_order=(0, 0, 0, 0), exog=exog_train)
fitted_model = model.fit(method='nm', maxiter=1000, disp=False)

# Summary of the model
print(fitted_model.summary())

# Plot the residuals to check for white noise
residuals = fitted_model.resid
plt.figure(figsize=(10, 6))
plt.plot(residuals)
plt.title('Residuals')
plt.show()

plot_acf(residuals, lags=25)
plt.show()

plot_pacf(residuals, lags=25)
plt.show()

# 
# Forecast
forecast = fitted_model.get_forecast(steps=len(test), exog=exog_test)
mean_forecast = forecast.predicted_mean
confidence_intervals = forecast.conf_int()

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(train.index, train['meter_sale_price'], label='Training Data', color='blue')
plt.plot(test.index, test['meter_sale_price'], label='Test Data', color='green')
plt.plot(mean_forecast.index, mean_forecast, label='Forecasted Data', color='red')
plt.fill_between(mean_forecast.index, 
                 confidence_intervals.iloc[:, 0], 
                 confidence_intervals.iloc[:, 1], 
                 color='pink', alpha=0.3)
plt.title('SARIMAX Model Forecast vs Actual Data')
plt.xlabel('Date')
plt.ylabel('Meter Sale Price')
plt.legend()
plt.show()

In [ ]:
from statsmodels.sandbox.regression.gmm import IV2SLS
from scipy.stats import chi2

# Hausman Test
# First stage regression of endogenous variable on instruments
instruments = sm.add_constant(train[['gdp']])

# Модель OLS
ols_model = sm.OLS(train['meter_sale_price'], exog_train).fit()

# Модель IV (инструментальные переменные)
iv_model = IV2SLS(train['meter_sale_price'], exog_train, instrument=instruments).fit()

# Тест Хаусмана вручную
b_ols = ols_model.params
b_iv = iv_model.params

# Найдем разницу между коэффициентами
diff = b_ols - b_iv

# Дисперсии коэффициентов
cov_ols = ols_model.cov_params()
cov_iv = iv_model.cov_params()

# Разница в дисперсиях
cov_diff = cov_ols - cov_iv

# Статистика Хаусмана
hausman_stat = diff @ np.linalg.inv(cov_diff) @ diff.T
p_value = 1 - chi2.cdf(hausman_stat, df=diff.shape[0])

print('Hausman Test Statistic:', hausman_stat)
print('P-value:', p_value)

In [ ]:
# Предсказание на том же временном периоде
pred = fitted_model.get_prediction(start=train.index[0], steps=len(train), exog=exog_train, dynamic=False)
pred_ci = pred.conf_int()

# Сравнение фактических и предсказанных значений
y_forecasted = pred.predicted_mean
y_truth = train['meter_sale_price']

mean_actual = y_truth.mean()

# Расчет метрик точности
mae = mean_absolute_error(y_truth, y_forecasted)
mse = mean_squared_error(y_truth, y_forecasted)
rmse = np.sqrt(mse)

relative_mae = (mae / mean_actual) * 100
relative_mse = (mse / (mean_actual ** 2)) * 100
relative_rmse = (rmse / mean_actual) * 100

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

print(f'Relative MAE (%): {relative_mae:.2f}%')
print(f'Relative MSE (%): {relative_mse:.2f}%')
print(f'Relative RMSE (%): {relative_rmse:.2f}%')

In [ ]:
ax = train['meter_sale_price'].plot(label='Observed', figsize=(14, 7))
y_forecasted.plot(ax=ax, label='Forecast', alpha=.7)
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Values')
plt.legend()
plt.show()

In [ ]:
train = sarimax_data[[
    'meter_sale_price', 
    'rooms', 
    'has_parking', 
    'distance_to_center', 
    'area_rating', 
    'gdp', 
]].iloc[-24:]

exog_train = train.drop('meter_sale_price', axis=1)

model = SARIMAX(train['meter_sale_price'], order=(1, 1, 1), seasonal_order=(0, 0, 0, 0), exog=exog_train)
fitted_model = model.fit(method='nm', maxiter=1000)

last_6_months = sarimax_data.tail(12)

future_6_months_dates = pd.date_range(start=train.index.max(), periods=12, freq='ME')


future_data = pd.DataFrame({
    'rooms': last_6_months['rooms'].tolist(),
    'has_parking': last_6_months['has_parking'].tolist(),
    'distance_to_center': last_6_months['distance_to_center'].tolist(),
    'area_rating': last_6_months['area_rating'].tolist(),
    'gdp': [527800000000.00 / 10 ** 9] * 12,
}, index=future_6_months_dates)

# 
forecast = fitted_model.get_forecast(steps=len(future_data), exog=future_data)
mean_forecast = forecast.predicted_mean
confidence_intervals = forecast.conf_int()

combined_data = pd.concat([train['meter_sale_price'], mean_forecast])

# Plotting
plt.figure(figsize=(12, 6))
plt.plot(combined_data.index, combined_data, label='Training Data', color='blue')
plt.plot(mean_forecast.index, mean_forecast, label='Forecasted Data', color='red')
plt.fill_between(mean_forecast.index, 
                 confidence_intervals.iloc[:, 0], 
                 confidence_intervals.iloc[:, 1], 
                 color='pink', alpha=0.3)
plt.title('SARIMAX Model Forecast from 2024-05')
plt.xlabel('Date')
plt.ylabel('Meter Sale Price')
plt.legend(loc='upper left')
plt.show()

mean_forecast.to_csv('predicted_residential_12m.csv')

In [ ]:
import scipy.stats as stats
from tabulate import tabulate

mean_price = np.mean(mean_forecast)
std_dev_price = np.std(mean_forecast)
skewness_price = stats.skew(mean_forecast)
kurtosis_price = stats.kurtosis(mean_forecast)
jarque_bera_test = stats.jarque_bera(mean_forecast)
beta = np.polyfit(np.arange(len(mean_forecast)), mean_forecast, 1)[0]

# Вычисление p-value и t-value для beta
slope, intercept, r_value, p_value, std_err = stats.linregress(np.arange(len(mean_forecast)), mean_forecast)
t_value = slope / std_err

# Создание DataFrame для отображения результатов
stat_analysis = pd.DataFrame({
    'Metric': ['Mean', 'Standard Deviation', 'Skewness', 'Kurtosis', 'Jarque-Bera', 'Beta', 'p-value', 't-value'],
    'Value': [mean_price, std_dev_price, skewness_price, kurtosis_price, jarque_bera_test[0], beta, p_value, t_value]
})

# Используем tabulate для отображения данных
print(tabulate(stat_analysis, headers='keys', tablefmt='pretty'))